In [ ]:
import pyspark
sc = pyspark.SparkContext('local[*]')

In [ ]:
!rm -rf metastore_db/*.lck

from pyspark.sql import SQLContext
sqlc = SQLContext(sc)

## PCA

In [ ]:
!wget https://s3.eu-central-1.amazonaws.com/dsr-data/UScrime/UScrime2-colsLotsOfNAremoved.csv

crimes = sqlc.read.format("com.databricks.spark.csv") \
            .option("delimiter", ",") \
            .option("header", "true") \
            .option("inferSchema", "true") \
            .load("UScrime2-colsLotsOfNAremoved.csv")

In [ ]:
crimes = crimes.drop("OtherPerCap").drop("community")

In [ ]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.linalg import Vectors, Vector

In [ ]:
assembler = VectorAssembler().setInputCols(crimes.columns).setOutputCol("features")

In [ ]:
featuresDF = assembler.transform(crimes).select("features")

In [ ]:
from pyspark.ml.feature import PCA

pca = PCA(k=10, inputCol="features", outputCol="pca")

model = pca.fit(featuresDF)

pc = model.transform(featuresDF)

pc.toPandas()[:3]

In [ ]:
sc.stop()